# Astrostat Lab4: model selection

In [1]:
# run the original notebook for M0fix and M0
%run 'fit.ipynb'

/Users/larryli/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


119  HARPS points;  122  HIRES points
Initial run of M0fix: chi2 = 395.796125727, 241 points
Real M0 run: chi2 = 390.394636688, loglike = -567.77961823


In [2]:
from scipy.special import erf
import numpy as np

In [3]:
# define BIC, AIC, reduced chi2, bayes factor functions

def get_BIC(loglike, k, N):
    """calculates the BIC
    Arguments:
        loglike {float} -- log likelihood
        k {int} -- number of parameters
        N {int} -- number of data points
    Return:
        BIC {float} -- Bayesian Information Criterion
    """
    BIC = k * np.log(N) - 2 * loglike
    return BIC


def get_AIC(loglike, k):
    """calculates the AIC
    Arguments:
        loglike {float} -- log likelihood
        k {int} -- number of parameters
    Return:
        AIC {float} -- Akaike Information Criterion
    """
    AIC = 2 * k - 2 * loglike
    return AIC

def get_reduced_chi2(chi2, k, N):
    """calculates the reduced chi2
    Arguments:
        chi2 {float} -- log likelihood
        k {int} -- number of parameters
        N {int} -- number of data points
    Return:
        reduced_chi2 {float}
    """
    reduced_chi2 = chi2 / (N - k)
    return reduced_chi2

def get_bayes_factor(IC1, IC2):
    """calculates the bayes factor
    Arguments:
        IC1 {float} -- could be BIC or AIC for the first model
        IC2 {float} -- could be BIC or AIC for the second model
    return:
        bayes_factor {float} -- is > 1 if IC1 < IC2
    """
    bayes_factor = np.e ** (-.5 * (IC1 - IC2))
    return bayes_factor

In [4]:
# chi2 from model def
def chi2_model(data, model, best_fit_params):
    """calculates chi2 based on model and best fit
    Arguments:
        data {np.ndarray} -- real data
        model {function} -- model
        best_fit_params {list} -- of best fit values, must match model input params
    """
    chi2 = 0.0
    for i in range(len(data)):
        m_sol = model(data[i, 0], *best_fit_params)
        chi2 = chi2 + ((data[i, 1] - m_sol) / data[i,2])**2
    return chi2

### M0fix & M0 (coded up in fix.ipynb): planets b c d e

In [5]:
m0_N = 119 + 122 # HARPS and HIRES datapoints
m0_k = 12 # degrees of freedom
m0_chi2 = 390.531407575 # from fit.ipynb run
m0_loglike = -567.848003674 # from fit.ipynb run

In [6]:
m0_AIC = get_AIC(m0_loglike, m0_k)
m0_BIC = get_BIC(m0_loglike, m0_k, m0_N)
m0_rchi2 = get_reduced_chi2(m0_chi2, m0_k, m0_N)

### Mffix & Mf: planets b c d e + f

In [7]:
# 4 planet + f model with K & P fixed to Vogt+ reported values
def Mffix(t, qb, qc, qd, qe, qf):
    return Kb_sol*np.sin(2.0*np.pi*t/Pb_sol+qb)\
           + Kc_sol*np.sin(2.0*np.pi*t/Pc_sol+qc)\
           + Kd_sol*np.sin(2.0*np.pi*t/Pd_sol+qd)\
           + Ke_sol*np.sin(2.0*np.pi*t/Pe_sol+qe)\
           + Kf_sol*np.sin(2.0*np.pi*t/Pf_sol+qf)

# 4 planet + f full model
def Mf(t, Kb, Pb, qb, Kc, Pc, qc, Kd, Pd, qd, Ke, Pe, qe, Kf, Pf, qf):
    return Kb*np.sin(2.0*np.pi*t/Pb+qb)\
           + Kc*np.sin(2.0*np.pi*t/Pc+qc)\
           + Kd*np.sin(2.0*np.pi*t/Pd+qd)\
           + Ke*np.sin(2.0*np.pi*t/Pe+qe)\
           + Kf*np.sin(2.0*np.pi*t/Pf+qf)

In [8]:
# param bounds and initial guesses for Mffix
param_bounds_mffix = ([-2.*np.pi]*5, [2.*np.pi]*5)

initial_guess_mffix = [np.random.uniform(-np.pi,np.pi)]*5

In [9]:
# fit Mffix and get best fitting parameters and chi2
Mffix_best, Mffix_cov = curve_fit(Mffix, data[:,0], data[:,1], sigma=data[:,2],
                                  p0=initial_guess_mffix, bounds=param_bounds_mffix)

# Get the chi2
mffix_chi2 = chi2_model(data, Mffix, Mffix_best)

# Print output
print('Mffix: chi2 = {0}, {1} points'.format(mffix_chi2,len(data)))

Mffix: chi2 = 349.597443436, 241 points


In [10]:
# setting phase solutions as initial guesses for Mf
qb_sol_mffix, qc_sol_mffix, qd_sol_mffix, qe_sol_mffix, qf_sol_mffix = Mffix_best

In [11]:
# param bounds and initial guesses for Mf
param_bounds_mf = ([0.0,  Pb_sol-3.0*Pb_del, -2.0*np.pi,
                    0.0,  Pc_sol-3.0*Pc_del, -2.0*np.pi,
                    0.0,  Pd_sol-3.0*Pd_del, -2.0*np.pi,
                    0.0,  Pe_sol-3.0*Pe_del, -2.0*np.pi,
                    0.0,  Pf_sol-3.0*Pf_del, -2.0*np.pi],
                   [20.0, Pb_sol+3.0*Pb_del,  2.0*np.pi,
                    20.0, Pc_sol+3.0*Pc_del,  2.0*np.pi,
                    20.0, Pd_sol+3.0*Pd_del,  2.0*np.pi,
                    20.0, Pe_sol+3.0*Pe_del,  2.0*np.pi,
                    20.0, Pf_sol+3.0*Pf_del,  2.0*np.pi])

# mffix results as inputs
initial_guess_mf = [Kb_sol, Pb_sol, qb_sol_mffix,
                    Kc_sol, Pc_sol, qc_sol_mffix,
                    Kd_sol, Pd_sol, qd_sol_mffix,
                    Ke_sol, Pe_sol, qe_sol_mffix,
                    Kf_sol, Pf_sol, qf_sol_mffix]

In [12]:
# fit Mf and get best fitting parameters and chi2 and loglike
Mf_best, Mf_cov = curve_fit(Mf, data[:,0], data[:,1], sigma=data[:,2],
                            p0=initial_guess_mf, bounds=param_bounds_mf)

# Get the chi2
mf_chi2 = chi2_model(data, Mf, Mf_best)

# Get the loglike
mf_loglike = -0.5*len(data)*np.log(2.0*np.pi) - np.sum(np.log(data[:,2])) - 0.5*mf_chi2

# Print output
print('Mf run: chi2 = {0}, loglike = {1}'.format(mf_chi2,mf_loglike))

Mf run: chi2 = 344.95839291, loglike = -545.061496341


In [13]:
# AIC, BIC, reduced chi2 for Mf
mf_N = m0_N # same number of datapoints
mf_k = 15 # degrees of freedom

mf_AIC = get_AIC(mf_loglike, mf_k)
mf_BIC = get_BIC(mf_loglike, mf_k, mf_N)
mf_rchi2 = get_reduced_chi2(mf_chi2, mf_k, mf_N)

### Mgfix & Mg: planets b c d e + g

In [14]:
# 4 planet + g model with K & P fixed to Vogt+ reported values
def Mgfix(t, qb, qc, qd, qe, qg):
    return Kb_sol*np.sin(2.0*np.pi*t/Pb_sol+qb)\
           + Kc_sol*np.sin(2.0*np.pi*t/Pc_sol+qc)\
           + Kd_sol*np.sin(2.0*np.pi*t/Pd_sol+qd)\
           + Ke_sol*np.sin(2.0*np.pi*t/Pe_sol+qe)\
           + Kg_sol*np.sin(2.0*np.pi*t/Pg_sol+qg)

# 4 planet + g full model
# in reality this is the same model as Mf (a 4 sinusoid with no fixed parameters)
def Mg(t, Kb, Pb, qb, Kc, Pc, qc, Kd, Pd, qd, Ke, Pe, qe, Kg, Pg, qg):
    return Mf(t, Kb, Pb, qb, Kc, Pc, qc, Kd, Pd, qd, Ke, Pe, qe, Kg, Pg, qg)

In [15]:
# param bounds and initial guesses for Mffix
param_bounds_mgfix = ([-2.*np.pi]*5, [2.*np.pi]*5)

initial_guess_mgfix = [np.random.uniform(-np.pi,np.pi)]*5

In [16]:
# fit Mffix and get best fitting parameters and chi2
Mgfix_best, Mgfix_cov = curve_fit(Mgfix, data[:,0], data[:,1], sigma=data[:,2],
                                  p0=initial_guess_mgfix, bounds=param_bounds_mgfix)

# Get the chi2
mgfix_chi2 = chi2_model(data, Mgfix, Mgfix_best)

# Print output
print('Mgfix: chi2 = {0}, {1} points'.format(mgfix_chi2,len(data)))

Mgfix: chi2 = 347.861935827, 241 points


In [17]:
# setting phase solutions as initial guesses for Mg
qb_sol_mgfix, qc_sol_mgfix, qd_sol_mgfix, qe_sol_mgfix, qg_sol_mgfix = Mgfix_best

In [18]:
# param bounds and initial guesses for Mg
# the first 4 bounds are the same as for Mf
param_bounds_mg = (param_bounds_mf[0][:-3] + [0.0, Pg_sol-3.0*Pg_del, -2.0*np.pi],
                   param_bounds_mf[1][:-3] + [20.0, Pg_sol+3.0*Pg_del,  2.0*np.pi])

# mgfix results as inputs
initial_guess_mg = [Kb_sol, Pb_sol, qb_sol_mgfix,
                    Kc_sol, Pc_sol, qc_sol_mgfix,
                    Kd_sol, Pd_sol, qd_sol_mgfix,
                    Ke_sol, Pe_sol, qe_sol_mgfix,
                    Kg_sol, Pg_sol, qg_sol_mgfix]

In [19]:
# fit Mg and get best fitting parameters and chi2 and loglike
Mg_best, Mg_cov = curve_fit(Mg, data[:,0], data[:,1], sigma=data[:,2],
                            p0=initial_guess_mg, bounds=param_bounds_mg)

# Get the chi2
mg_chi2 = chi2_model(data, Mg, Mg_best)

# Get the loglike
mg_loglike = -0.5*len(data)*np.log(2.0*np.pi) - np.sum(np.log(data[:,2])) - 0.5*mg_chi2

# Print output
print('Mg run: chi2 = {0}, loglike = {1}'.format(mg_chi2, mg_loglike))

Mg run: chi2 = 341.941846866, loglike = -543.553223319


In [20]:
# AIC, BIC, reduced chi2 for Mg
mg_N = m0_N # same number of datapoints
mg_k = 15 # degrees of freedom

mg_AIC = get_AIC(mg_loglike, mg_k)
mg_BIC = get_BIC(mg_loglike, mg_k, mg_N)
mg_rchi2 = get_reduced_chi2(mg_chi2, mg_k, mg_N)

### Mfgfix & Mfg: planets b c d e + f&g

In [21]:
# 4 planet + f + g model with K & P fixed to Vogt+ reported values
def Mfgfix(t, qb, qc, qd, qe, qf, qg):
    return Kb_sol*np.sin(2.0*np.pi*t/Pb_sol+qb)\
           + Kc_sol*np.sin(2.0*np.pi*t/Pc_sol+qc)\
           + Kd_sol*np.sin(2.0*np.pi*t/Pd_sol+qd)\
           + Ke_sol*np.sin(2.0*np.pi*t/Pe_sol+qe)\
           + Kf_sol*np.sin(2.0*np.pi*t/Pf_sol+qf)\
           + Kg_sol*np.sin(2.0*np.pi*t/Pg_sol+qg)

# 4 planet + f + g full model
def Mfg(t, Kb, Pb, qb, Kc, Pc, qc, Kd, Pd, qd, Ke, Pe, qe, Kf, Pf, qf, Kg, Pg, qg):
    return Kb*np.sin(2.0*np.pi*t/Pb+qb)\
           + Kc*np.sin(2.0*np.pi*t/Pc+qc)\
           + Kd*np.sin(2.0*np.pi*t/Pd+qd)\
           + Ke*np.sin(2.0*np.pi*t/Pe+qe)\
           + Kf*np.sin(2.0*np.pi*t/Pf+qf)\
           + Kg*np.sin(2.0*np.pi*t/Pg+qg)

In [22]:
# param bounds and initial guesses for Mfgfix
param_bounds_mfgfix = ([-2.*np.pi]*6, [2.*np.pi]*6)

initial_guess_mfgfix = [np.random.uniform(-np.pi,np.pi)]*6

In [23]:
# fit Mfgfix and get best fitting parameters and chi2
Mfgfix_best, Mfgfix_cov = curve_fit(Mfgfix, data[:,0], data[:,1], sigma=data[:,2],
                                  p0=initial_guess_mfgfix, bounds=param_bounds_mfgfix)

# Get the chi2
mfgfix_chi2 = chi2_model(data, Mfgfix, Mfgfix_best)

# Print output
print('Mfgfix: chi2 = {0}, {1} points'.format(mfgfix_chi2,len(data)))

Mfgfix: chi2 = 302.861634463, 241 points


In [24]:
# setting phase solutions as initial guesses for Mfg
qb_sol_mfgfix, qc_sol_mfgfix, qd_sol_mfgfix, qe_sol_mfgfix, qf_sol_mfgfix, qg_sol_mfgfix = Mfgfix_best

In [25]:
# param bounds and initial guesses for Mfg
# the first 5 bounds are the same as for Mf
param_bounds_mfg = (param_bounds_mf[0] + [0.0, Pg_sol-3.0*Pg_del, -2.0*np.pi],
                   param_bounds_mf[1] + [20.0, Pg_sol+3.0*Pg_del,  2.0*np.pi])

# mgfix results as inputs
initial_guess_mfg = [Kb_sol, Pb_sol, qb_sol_mfgfix,
                    Kc_sol, Pc_sol, qc_sol_mfgfix,
                    Kd_sol, Pd_sol, qd_sol_mfgfix,
                    Ke_sol, Pe_sol, qe_sol_mfgfix,
                    Kf_sol, Pf_sol, qf_sol_mfgfix,
                    Kg_sol, Pg_sol, qg_sol_mfgfix]

In [26]:
# fit Mg and get best fitting parameters and chi2 and loglike
Mfg_best, Mfg_cov = curve_fit(Mfg, data[:,0], data[:,1], sigma=data[:,2],
                              p0=initial_guess_mfg, bounds=param_bounds_mfg)

# Get the chi2
mfg_chi2 = chi2_model(data, Mfg, Mfg_best)

# Get the loglike
mfg_loglike = -0.5*len(data)*np.log(2.0*np.pi) - np.sum(np.log(data[:,2])) - 0.5*mfg_chi2

# Print output
print('Mg run: chi2 = {0}, loglike = {1}'.format(mfg_chi2, mfg_loglike))

Mg run: chi2 = 297.629779894, loglike = -521.397189833


In [27]:
# AIC, BIC, reduced chi2 for Mfg
mfg_N = m0_N # same number of datapoints
mfg_k = 18 # degrees of freedom

mfg_AIC = get_AIC(mfg_loglike, mfg_k)
mfg_BIC = get_BIC(mfg_loglike, mfg_k, mfg_N)
mfg_rchi2 = get_reduced_chi2(mfg_chi2, mfg_k, mfg_N)

### Simple model comparison

In [28]:
# looking at results
print('Chi2:\n\tM0: {0} \tMf: {1} \tMf: {2} \tMfg: {3}'
      .format(m0_chi2, mf_chi2, mg_chi2, mfg_chi2))
print('Reduced chi2:\n\tM0: {0} \tMf: {1} \tMf: {2} \tMfg: {3}'
      .format(m0_rchi2, mf_rchi2, mg_rchi2, mfg_rchi2))
print('BIC:\n\tM0: {0} \tMf: {1} \tMf: {2} \tMfg: {3}'
      .format(m0_BIC, mf_BIC, mg_BIC, mfg_BIC))
print('AIC:\n\tM0: {0} \tMf: {1} \tMf: {2} \tMfg: {3}'
      .format(m0_AIC, mf_AIC, mg_AIC, mfg_AIC))

print('\nChi2 and reduced chi2 do go down as we go to more complex models, but are they really better?')

Chi2:
	M0: 390.531407575 	Mf: 344.95839291 	Mf: 341.941846866 	Mfg: 297.629779894
Reduced chi2:
	M0: 1.70537732566 	Mf: 1.5263645704 	Mf: 1.51301702153 	Mfg: 1.33466269011
BIC:
	M0: 1201.51357055 	Mf: 1172.39494668 	Mf: 1169.37840064 	Mfg: 1141.52072447
AIC:
	M0: 1159.69600735 	Mf: 1120.12299268 	Mf: 1117.10644664 	Mfg: 1078.79437967

Chi2 and reduced chi2 do go down as we go to more complex models, but are they really better?


In [29]:
# Bayes factor from BIC
b_00_bic = bayes_factor(m0_BIC, m0_BIC)
b_0f_bic = bayes_factor(m0_BIC, mf_BIC)
b_0g_bic = bayes_factor(m0_BIC, mg_BIC)
b_0fg_bic = bayes_factor(m0_BIC, mfg_BIC)

print('Bayes factor approximated with the BIC:\n\tB_00: {0} \tB_0f: {1} \tB_0g: {2} \tB_0fg: {3}'
      .format(b_00_bic, b_0f_bic, b_0g_bic, b_0fg_bic))

NameError: name 'bayes_factor' is not defined

In [ ]:
# Bayes factor from AIC
b_00_aic = bayes_factor(m0_AIC, m0_AIC)
b_0f_aic = bayes_factor(m0_AIC, mf_AIC)
b_0g_aic = bayes_factor(m0_AIC, mg_AIC)
b_0fg_aic = bayes_factor(m0_AIC, mfg_AIC)

print('Bayes factor approximated with the AIC:\n\tB_00: {0} \tB_0f: {1} \tB_0g: {2} \tB_0fg: {3}'
      .format(b_00_aic, b_0f_aic, b_0g_aic, b_0fg_aic))

In [187]:
# in terms of sigmas...
# from BIC
p_b_00_bic = 1. / (1 + b_00_bic)
p_b_0f_bic = 1. / (1 + b_0f_bic)

In [192]:
1 - erf(p_b_00_bic)

0.47950012218695348

In [193]:
1 - erf(p_b_0f_bic)

0.1572994655191936

In [201]:
erf(3*0.7071067812)

0.99730020393724572

### Cross validation

In [359]:
# leave p out def
def select_training_validation_sets(data, p):
    """select the training set and the validation set with leave p-out method
    Arguments:
        data {np.ndarray} -- of full dataset (assuming 2d for now)
        p {int} -- number of datapoints to leave out
    Return:
        t_data {np.ndarray} -- of training set
        v_data {np.ndarray} -- of validation set
    """
    num_data_points, num_data_dim = data.shape
    assert p < .33 * num_data_points, 'Leaving out too many data points in leave p-out!'
    
    # placeholder data stacks
    t_set_list = []
    v_set_list = []

    # p choice indices set
    p_set = set(np.random.choice(range(num_data_points), p, replace=False))

    # assign data rows to data stacks based on if row is in p choice indices set
    for r in range(num_data_points):
        row = data[r,:]
        if r in p_set:
            v_set_list.append(row)
        else:
            t_set_list.append(row)
        
    t_data = np.vstack(tuple(t_set_list))
    v_data = np.vstack(tuple(v_set_list))
    return t_data, v_data